
Enter Team Member Names here (double click to edit):

- Name 1:
- Name 2:
- Name 3:

________

# In Class Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook) before the end of class.

________________________________________________________________________________________________________

## Loading the Classification Data
Please run the following code to read in the "digits" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import load_digits
import numpy as np

ds = load_digits()

# this holds the continuous feature data
print ('features shape:', ds.data.shape) # there are 1797 instances and 64 features per instance
print ('target shape:', ds.target.shape) 
print ('range of target:', np.min(ds.target),np.max(ds.target))

features shape: (1797, 64)
target shape: (1797,)
range of target: 0 9


________________________________________________________________________________________________________

## Using Decision Trees
In the videos, we talked about the splitting conditions for different attributes. Specifically, we discussed the number of ways in which it is possible to split a node, depending on the attribute types. To understand the possible splits, we need to understand the attributes. You might find the description in the `ds['DESCR']` field to be useful. You can see the field using `print ds['DESCR'] `

**Question 1:** For the digits dataset, what are the type(s) of the attributes? How many attributes are there? What do they represent?


In [4]:
## Enter your comments here

print ds['DESCR']
# Each attribute is a pixel from an 8x8 pixel array. They are images of handwritten digits. 
# The features are each pixel location in the 8x8 grid, thus there are 64 of them. The pixels are encoded as 
# 16 bit data, but are best considered as continuous attributes because they can take on many
# distinct values. 

## Enter comments here

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

___
## Using the gini coefficient
We talked about the gini index in the videos.  The gini coefficient for a given split is given by:
$$Gini=\sum_{t=1}^T \frac{n_t}{N}gini(t)$$
where $T$ is the total number splits (2 for binary), $n_t$ is the number of instances in node $t$ after splitting, and $N$ is the total number of instances in the parent node. $gini(t)$ is the gini index for the resulting node and is given by:
$$gini(t)=1-\sum_{j=0}^{C-1} p(j|t)^2$$
where $C$ is the total number of possible classes and $p(j|t)$ is the count of instances belonging to class $j$ in node $t$, normalized by the total number of instances in node $t$ (i.e., the probability of the class in node $t$).

For the given dataset, $gini(t)$ has been programmed for you in the function `gini_index`. To use the function, pass in an array of the classes for a given node and the gini will be returned. In the example below, the function is used calculate the gini for splitting the dataset on feature 28, with value 2.5. First all the target classes in a split are calculated using `ds.target[feature28>2.5]` and those classes are passed into the function to get the gini for the going right in the tree. Then the gini is calculated for going left in the tree. 

In [2]:
# compute the gini of several examples for the starting dataset

def gini_index(classes_in_split):
    # pay no attention to this code -- it just computes the gini for a given split 
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    unique_classes = np.unique(classes_in_split)
    gini = 1
    for c in unique_classes:
        gini -= (np.sum(classes_in_split==c) / float(len(classes_in_split)))**2
        
    return gini
    
# get the value for this feature as a column vector 
# (this is like grabbing one column of the record table)
feature28 = ds.data[:,28]

# if we split on the value of 2.5, then this is the gini for each resulting node:
gini_r = gini_index(ds.target[feature28>2.5])
gini_l = gini_index(ds.target[feature28<=2.5])

# compute gini example. This splits on attribute '28' with a value of 2.5
print 'gini for right node of split:', gini_r
print 'gini for left node of split:', gini_l

gini for right node of split: 0.884585786767
gini for left node of split: 0.711540756654


**Question 2:** Now, using the above values `gini_r` and `gini_l`. Calculate the combined Gini for the entire split. You will need to write the weighted summation (based upon the number of instances inside each node). To count the number of instances greater than a value using numpy, you can write `sum(some_array>5)` where `some_array` is an array of values and we want to now how many are greater than `5`.

In [3]:
## Enter your code here

# we need to make a weighted sum of the gini indices
num_instances_r = float(sum(feature28>2.5))
num_instances_l = float(sum(feature28<=2.5))
N = float(len(ds.target))

gini_total = (num_instances_r*gini_r + num_instances_l*gini_l) / N

print ('total gini is:', gini_total)

## Enter your code here

('total gini is:', 0.84616343450451792)


___
**Question 3:** Now we want to know which is a better split, `feature28 -> 2.5` or `feature28 -> 10`.  Enter your code to find the total gini of splitting on the threshold of 10 and compare it to the threshold for 2.5 for feature 28 (saved to variable `feature28`). According to gini, which threshold is better, 2.5 or 10.0?

In [4]:
# Enter your code here

# we need to make a weighted sum of the gini indices
T = 10
num_instances_r = sum(feature28>T)
num_instances_l = sum(feature28<=T)

gini_total = (num_instances_r*gini_index(ds.target[feature28>T]) + 
              num_instances_l*gini_index(ds.target[feature28<=T])) / float(len(ds.target))

print 'total gini for new threshold of 10.0 is:', gini_total
print 'This is larger than our previous gini and lower is better. Therefore, 2.5 is more optimal.'

# Enter your code here

total gini for new threshold of 10.0 is: 0.863611174323
This is larger than our previous gini and lower is better. Therefore, 2.5 is more optimal.


___
## Entropy based splitting
We discussed entropy as well in the video as another means of splitting. We calculated entropy for a node $t$ by:
$$ Entropy(t) = -\sum p(j|t) \log p(j|t) $$
where $p(j|t)$ is the same as above. To combine Entropy measures from a set of nodes, t = {1,...,T} we use: 
$$Entropy_{Total}=\sum_{t=1}^T \frac{n_t}{N}Entropy(t)$$ 
and information gain is calculated by subtracting the Entropy of the parent, with the Entropy of the split. You are given an equation for calculating the $Entropy(t)$ of  node $t$. It works exactly like the `gini_index` function above, but is named `entropy_value`. 

In [5]:
def entropy_value(classes_in_split):
    # pay no attention to this code -- it just computes the gini for a given split 
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    unique_classes = np.unique(classes_in_split)
    ent = 0
    for c in unique_classes:
        p = (np.sum(classes_in_split==c) / float(len(classes_in_split)))
        ent += p * np.log(p)
        
    return -ent

ent_r = entropy_value(ds.target[feature28>2.5])
ent_l = entropy_value(ds.target[feature28<=2.5])

# compute entropy example. This splits on attribute '28' with a value of 2.5
print 'entropy for right of split:', ent_r
print 'entropy for left of split:', ent_l

entropy for right of split: 2.18369753782
entropy for left of split: 1.48988814128


___
**Question 4:** Calculate the information gain of the split when the threshold is 2.5 on `feature28`. What is the value of the information gain?

In [6]:
# Enter your code here

# we need to make a weighted sum of the gini indices
T = 2.5
num_instances_r = sum(feature28>T)
num_instances_l = sum(feature28<=T)

ent2_5 = entropy_value(ds.target) - (num_instances_r*entropy_value(ds.target[feature28>T]) + 
              num_instances_l*entropy_value(ds.target[feature28<=T])) / float(len(ds.target))

print 'Information gain is:', ent2_5

# Enter your code here

Information gain is: 0.272832851327


**Question 5:** What is the information gain if the threshold is 10.0 on `feature28`? According to information gain, is it better to split on a threshold of 2.5 or 10? Does entropy give the same decision as gini for this example?

In [7]:
# Enter your code here

# we need to make a weighted sum of the gini indices
T = 10
num_instances_r = sum(feature28>T)
num_instances_l = sum(feature28<=T)

ent10 = entropy_value(ds.target) - (num_instances_r*entropy_value(ds.target[feature28>T]) + 
              num_instances_l*entropy_value(ds.target[feature28<=T])) / float(len(ds.target))


print num_instances_l
print num_instances_r
print 'Information gain is:', ent10
print 'The gain is smaller, meaning that the entropy dropped less. The threshold of 2.5 is better!'
print 'That is the same conclusion as the gini!'

# Enter your code here

754
1043
Information gain is: 0.209551377044
The gain is smaller, meaning that the entropy dropped less. The threshold of 2.5 is better!
That is the same conclusion as the gini!


___
## Information gain and multi-way splitting
Now assume that we can use not just a binary split, but a three way split. 

**Question 6** What is the information gain if we split feature28 on two thesholds (three separate nodes corresponding to three branches from one node) 
- node left: `feature28<2.5`, 
- node middle: `2.5<=feature28<10`, and 
- node right: `10<=feature28`? 

Is the information gain better? 

***Note***: You can index into a numpy array with the following notation: `some_array[(2.5<=feature28) & (feature28<10.0)]`

In [8]:
# Enter your code here

# we need to make a weighted sum of the gini indices
T1 = 2.5
T2 = 10
num_instances_r = sum(feature28>10)
num_instances_l = sum(feature28<=2.5)
num_instances_m = len(ds.target) - num_instances_r - num_instances_l
# Or you can also calculate it this way!
# num_instances_m = sum((T1<feature28) & (feature28<=T2))

ent_multi = entropy_value(ds.target) - (num_instances_r*entropy_value(ds.target[feature28>T2]) + 
              num_instances_l*entropy_value(ds.target[feature28<=T1]) +
              num_instances_m*entropy_value(ds.target[(T1<=feature28) & (feature28<T2)])) / float(len(ds.target))

print 'Information gain is:', ent_multi
print 'The gain is better! That means the three way split is "better" than the two way split.'
print 'Or does it? We need to normalize by split info!!'

# Enter your code here

Information gain is: 0.31972399967
The gain is better! That means the three way split is "better" than the two way split.
Or does it? We need to normalize by split info!!


**Question 7**: Should we normalize the quantity that we just calculated if we want to compare it to the information gain of a binary split? Why or Why not?

In [9]:
# Enter your code here

# The Short answer is Yes! We need to normalize the value by the split info so that we do not
# Always prefer the larger, purer splits.

# based on email sent by Prof Jake Drew on 2/15/2018 at Thu, Feb 15, 2018 at 8:51 PM --> benbroc@gmail.com
# On question #7 students are not required to write code just mention that they should normalize 
# to keep the algorithm from favoring the purer, larger splits.

# if we normalize by the entropy of the splits we get:
tmp_split = np.zeros(ds.target.shape)
tmp_split[feature28>T2] = 1
tmp_split[feature28<=T1] = 2
split_entropy = entropy_value(tmp_split)

gain_ratio = ent_multi / split_entropy
print 'gain ratio of multi-split is actually:', gain_ratio

# gain ratio of 2.5 
tmp_split = np.zeros(ds.target.shape)
tmp_split[feature28>2.5] = 1
gain_ratio_T1 = ent2_5 / entropy_value(tmp_split)
print 'gain ratio of 2.5:',  gain_ratio_T1
print 'That means the 2.5 threshold is still more optimal than the multiway split, once normalized!!'

# Enter your code here

gain ratio of multi-split is actually: 0.329517155767
gain ratio of 2.5: 0.515290639568
That means the 2.5 threshold is still more optimal than the multiway split, once normalized!!


___
## Decision Trees in scikit-learn
Scikit-learn also has an implementation of decision trees. Its available here:
- http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
- http://scikit-learn.org/stable/modules/tree.html#tree

**Question 8**: What algorithm does scikit-learn use for creating decision trees (i.e., ID3, C4.5, C5.0, CART, MARS, CHAID, etc.)? 

In [11]:
# Enter your answer here

print 'CART'
# http://scikit-learn.org/stable/modules/tree.html

# Enter your answer here

CART


___
**Question 9**: Using the documentation, use scikit-learn to train a decision tree on the digits data. Calculate the accuracy on the training data. What is the accuracy? Did you expect the decision tree to have this kind of accuracy? Why or Why not?

In [10]:
# use scikit learn to train a decision tree
from sklearn.tree import DecisionTreeClassifier

# enter your code below here to train and predict on the same data
tr = DecisionTreeClassifier(criterion='gini')
tr.fit(ds.data,ds.target)
yhat = tr.predict(ds.data)
# enter your code above here

from sklearn.metrics import accuracy_score 

# enter your code below here
print 'accuracy:', accuracy_score(ds.target,yhat)
print 'Decision trees can learn any function, so of course they have perfect accuracy on the training data'
print 'Its overfit though--we would need a validation set to really test out the accuracy'
# enter your code above here


accuracy: 1.0
Decision trees can learn any function, so of course they have perfect accuracy on the training data
Its overfit though--we would need a validation set to really test out the accuracy


___
**Question 10**: Look at the other input variables to the function `DecisionTreeClassifier` could any of them be used to help prevent the decision tree from overlearning on the data? Which variables might we use to control overfitting and how (explain each)?

In [13]:
# Enter your answer here

#  Refereence link --> http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
#
# Here are all the available inputs to the function: 
# DecisionTreeClassifier(criterion='gini', splitter='best', 
# max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, 
# random_state=None, min_density=None, max_leaf_nodes=None)¶

print 'We could make "max_depth" smaller to prevent long branches'
print 'We could increase "min_samples_split" to prevent small boxes forming decision boundaries'
print 'We could increase the "min_samples_per_leaf" to control small boxes forming'
print 'We could decrease the "max_leaf_nodes" to control how many boxes can be formed in feature space'


# Enter your answer here

We could make "max_depth" smaller to prevent long branches
We could increase "min_samples_split" to prevent small boxes forming decision boundaries
We could increase the "min_samples_per_leaf" to control small boxes forming
We could decrease the "max_leaf_nodes" to control how many boxes can be formed in feature space


________________________________________________________________________________________________________

That's all! Please **upload your rendered ** and please include **team member names** in the notebook submission.